In [ ]:
import sys,os
import numpy as np
lib_path=os.path.abspath("/home/ferrigno/Soft/pysas")
if lib_path not in sys.path:
    sys.path.append(lib_path)
else:
    print("Not appending")
    

import pysas

In [ ]:
from astropy.io import fits as pf
def set_rsp_syst(s,r,a,sys):
    f_f  = pf.open(s, 'update')
    f_f[1].header['RESPFILE']=r
    f_f[1].header['ANCRFILE']=a
    f_f[1].data['SYS_ERR']=sys
    f_f.flush()
    f_f.close()

In [ ]:

for year in range(2003,2004):
    i_spec_file='Crab_%d_spectrum_osa10.fits'%year
    i_rmf_file='Crab_%d_rmf_osa10.fits.gz'%year
    i_arf_file='Crab_%d_arf_osa10.fits.gz'%year
    
    i_sys=0.02
    
    set_rsp_syst(i_spec_file, i_rmf_file, i_arf_file, i_sys)
    
    j_spec_file='Crab_%d_spectrum_osa10_jemx1.fits'%year
    j_rmf_file='Crab_%d_rmf_osa10_jemx1.fits.gz'%year
    j_arf_file='Crab_%d_arf_osa10_jemx1.fits.gz'%year
    
    j_sys=0.02
    
    set_rsp_syst(j_spec_file, j_rmf_file, j_arf_file, j_sys)
    
    
    
    

In [ ]:
import xspec
verbose=True
mod_file='mod_2bknpow.xcm'
verbose=True


chains=[]
fit_by_bin={}

for year in range(2003,2019):

    isgri_spec='Crab_%d_spectrum_osa10.fits'%year
    jemx1_spec='Crab_%d_spectrum_osa10_jemx1.fits'%year
    outputfiles_basename='%d'%year+'-'+mod_file.replace('mod_','').replace('.xcm','')+"-"
    print(outputfiles_basename)
    run_chain=False
    load_chain=False
    perform_fit=True
    ignore_string=['**-20.0, 300.0-**', '**-3.5,20.0-**']
    if year == 2003:
        ignore_string=['**-30.0, 300.0-**', '**-3.5,20.0-**']
    chain_name, fit_res =pysas.epic_xspec_mcmc_fit(xspec, mod_file, 
                                outputfiles_basename = outputfiles_basename,
                                pn_spec = isgri_spec,
                                mos1_spec = jemx1_spec,
                                mos2_spec = 'none', 
                                jeffreys_priors=[],
                                ignore_string=ignore_string,
                                load_chain=load_chain, perform_fit=True, set_priors=True, walkers=40, 
                                               run_chain=run_chain,
                                               compute_errors=True, save_xcm=True, statistics='chi' )
    chains.append(chain_name)
    
    #exposure, tstart, tstop = pysas.get_spec_exp_times(ss)
    
    #print(tstart, tstop)
    
    exposure, tstart, tstop = pysas.get_spec_exp_times(isgri_spec)
    fit_res.update( {'times': [tstart,tstop] } )
        
    fit_by_bin.update({outputfiles_basename : fit_res})

In [ ]:
fit_by_bin

In [ ]:
latex_label_dict = {
    'cstat' : '$\\chi^2_\\mathrm{red}$',
    'lg10Flux__03' : '$\\log(F_\\mathrm{20-100keV})$',
    'lg10Flux__12' : '$\\log(F_\\mathrm{5-20keV})$',
    'PhoIndx1__04' : '$\\Gamma_1$',
    'PhoIndx2__06' : '$\\Gamma_2$',
    'PhoIndx3__08' : '$\\Gamma_3$'
}

In [ ]:
pysas.plot_fit_parameters_norate(fit_by_bin, plot_latex=True, latex_label_dict=latex_label_dict,
                               skipped=['factor', 'plot_filename'], save_plot=True,
                               xlabel='Time [MJD-51544]', title='Crab', log_scale_labels=[])

In [ ]:
!pdfcrop spec_results_Crab.pdf
!mv spec_results_Crab-crop.pdf spec_results_Crab.pdf


In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter

plt.figure(figsize=(10,7))

for year in range(2003,2019):
    outputfiles_basename='%d'%year+'-'+mod_file.replace('mod_','').replace('.xcm','')+"-"
    print(outputfiles_basename)
    xspec.Xset.restore(outputfiles_basename+'model.xcm')
    xspec.Plot.device='/NULL'
    xspec.Plot('eeuf')
    xspec.Plot.setRebin(15,20)
    
    
    en=xspec.Plot.x(1)
    en_err=xspec.Plot.xErr(1)
    fl=xspec.Plot.y(1)
    fl_err=xspec.Plot.yErr(1)    
    mo=xspec.Plot.model(1)
    
    plt.errorbar(en,fl,xerr=en_err,yerr=fl_err,linestyle='none',linewidth=8,color='blue',alpha=0.1)
    plt.plot(en,mo,color='black',linewidth=4,alpha=0.1)
    
    en=xspec.Plot.x(2)
    en_err=xspec.Plot.xErr(2)
    fl=xspec.Plot.y(2)
    fl_err=xspec.Plot.yErr(2)
    mo=xspec.Plot.model(2)
    
    plt.errorbar(en,fl,xerr=en_err,yerr=fl_err,linestyle='none',linewidth=8,color='blue',alpha=0.1)
    plt.plot(en,mo,color='black',linewidth=4,alpha=0.1)
    


plt.tick_params(axis='both', which='major', labelsize=16)
plt.text(5,10,'JEMX-1',fontsize=16)
plt.text(50,8,'IBIS/ISGRI',fontsize=16)

plt.xscale('log')
plt.yscale('log')
plt.ylim(2,15.)
plt.xlim(3,300)
plt.xlabel('$E$ [keV]',fontsize=16)
plt.ylabel('$E^2F_E$ [keV cm$^{-2}$ s$^{-2}$)',fontsize=16)

ax=plt.gca()
ax.xaxis.set_major_formatter(FormatStrFormatter('%.0f'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.0f'))
ax.yaxis.set_minor_formatter(FormatStrFormatter('%.0f'))
ax.tick_params(axis='both', which='major', labelsize=16)
ax.tick_params(axis='both', which='minor', labelsize=16)

#plt.legend(loc='lower left',fontsize=16)
plt.savefig('Crab_spectrum1.pdf',format='pdf',dpi=100)